In [1]:
from tensorflow.keras.utils import to_categorical
import plotting
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

In [2]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 

del train 
X_train = X_train / 255.0
test = test / 255.0
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [4]:
model = Sequential()

model.add(Conv2D(name = 'conv1', filters = 8, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPool2D(name = 'max1', pool_size=(2,2)))
model.add(BatchNormalization(name = 'batch1'))
model.add(Activation('relu', name = 'relu1'))
model.add(Dropout(0.25, name = 'drop1'))

model.add(Conv2D(name = 'conv2', filters = 8, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(name = 'max2', pool_size=(2,2)))
model.add(BatchNormalization(name = 'batch2'))
model.add(Activation('relu', name = 'relu2'))
model.add(Dropout(0.25, name = 'drop2'))


model.add(Conv2D(name = 'conv3', filters = 8, kernel_size = (3,3),padding = 'Same'))
model.add(MaxPool2D(name = 'max3', pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization(name = 'batch3'))
model.add(Activation('relu', name = 'relu3'))
model.add(Dropout(0.25, name = 'drop3'))


model.add(Flatten())
# model.add(Dense(48, activation = "relu", name = 'dense1'))
# model.add(BatchNormalization(name = 'batch4'))
# model.add(Activation('relu', name = 'relu4'))
# model.add(Dropout(0.25, name = 'drop4'))

model.add(Dense(32, activation = "relu", name = 'dense2'))
model.add(BatchNormalization(name = 'batch5'))
model.add(Activation('relu', name = 'relu5'))
model.add(Dropout(0.25, name = 'drop5'))

model.add(Dense(10, name = 'dense5'))
model.add(Activation('softmax', name = 'soft'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 28, 28, 8)         80        
_________________________________________________________________
max1 (MaxPooling2D)          (None, 14, 14, 8)         0         
_________________________________________________________________
batch1 (BatchNormalization)  (None, 14, 14, 8)         32        
_________________________________________________________________
relu1 (Activation)           (None, 14, 14, 8)         0         
_________________________________________________________________
drop1 (Dropout)              (None, 14, 14, 8)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 14, 14, 8)         584       
_________________________________________________________________
max2 (MaxPooling2D)          (None, 7, 7, 8)          

In [6]:
model = Sequential()
model.add(Conv2D(4, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 4)         40        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 4)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 8)         296       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 8)           0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 16)          1168      
_________________________________________________________________
flatten_1 (Flatten)          (None, 144)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)               

In [7]:
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=32,
              epochs=20, validation_split=0.25, shuffle=True)

Epoch 1/20
886/886 [==============================] - 6s 7ms/step - loss: 1.8756 - accuracy: 0.3849 - val_loss: 1.0582 - val_accuracy: 0.7352
Epoch 2/20
886/886 [==============================] - 6s 7ms/step - loss: 0.7264 - accuracy: 0.7978 - val_loss: 0.5577 - val_accuracy: 0.8389
Epoch 3/20
886/886 [==============================] - 7s 7ms/step - loss: 0.5141 - accuracy: 0.8480 - val_loss: 0.4621 - val_accuracy: 0.8638
Epoch 4/20
886/886 [==============================] - 6s 7ms/step - loss: 0.4381 - accuracy: 0.8694 - val_loss: 0.4014 - val_accuracy: 0.8829
Epoch 5/20
886/886 [==============================] - 6s 7ms/step - loss: 0.3924 - accuracy: 0.8812 - val_loss: 0.3630 - val_accuracy: 0.8942
Epoch 6/20
886/886 [==============================] - 6s 7ms/step - loss: 0.3604 - accuracy: 0.8910 - val_loss: 0.3413 - val_accuracy: 0.8982
Epoch 7/20
886/886 [==============================] - 6s 7ms/step - loss: 0.3355 - accuracy: 0.8980 - val_loss: 0.3168 - val_accuracy: 0.9059
Epoch 

In [8]:
def accuracy(y_pred, y_true):
    cnt = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            cnt += 1
    return cnt/len(y_pred)
pred = model.predict(X_val)
y_pred = np.argmax(pred, axis = 1)
y_true = np.argmax(Y_val, axis = 1)
print(accuracy(y_pred, y_true))

0.9504761904761905


In [9]:
model.save('mnist_model3.h5')